<a href="https://colab.research.google.com/github/Jaeji/AM360Paper/blob/main/Paper_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1: ติดตั้งไลบรารีและระบบโหลดไฟล์ (Setup)

In [16]:
import os
import shutil
import pandas as pd
import numpy as np
import xgboost as xgb
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# =========================================================
# 🛠️ GITHUB CLONING PROCESS
# =========================================================
print("🚀 Initiating System Setup...")

# 1. ตั้งค่าชื่อ Repo และ URL ให้ถูกต้อง
repo_url = "https://github.com/Jaeji/AM360Paper.git"
repo_name = "AM360Paper"  # ⚠️ ชื่อโฟลเดอร์ที่ Git จะสร้าง (ต้องตรงกับชื่อหลัง slash สุดท้าย)

# 2. เคลียร์ของเก่า (เผื่อเคย Clone ผิดไว้)
if os.path.exists(repo_name):
    shutil.rmtree(repo_name)

# 3. สั่ง Clone
print(f"📡 Cloning from: {repo_url}...")
!git clone {repo_url}

# 4. ย้ายไฟล์ (หาจากหน้าแรกของโฟลเดอร์เลย)
source_path = f"{repo_name}/B0005.mat"  # 👈 แก้ตรงนี้: ไม่ต้องมี /data/ แล้ว
destination = "B0005.mat"

if os.path.exists(source_path):
    # ย้ายไฟล์มาวางหน้าสุด
    shutil.move(source_path, destination)
    print(f"✅ เจอไฟล์แล้ว! ย้าย {destination} ออกมาเรียบร้อย")

    # ลบโฟลเดอร์ทิ้งเพื่อความสะอาด
    shutil.rmtree(repo_name)
    print("🧹 Cleanup complete.")

else:
    print(f"❌ Error: หาไฟล์ไม่เจอที่ {source_path}")
    print("   (ลองเช็คดูว่าใน Git ชื่อไฟล์ B0005.mat ตัวเล็กตัวใหญ่ตรงกันไหม?)")

print("\n✅ Setup Complete: พร้อมรัน Section ถัดไป")

🚀 Initiating System Setup...
📡 Cloning from: https://github.com/Jaeji/AM360Paper.git...
Cloning into 'AM360Paper'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 15.21 MiB | 17.52 MiB/s, done.
Resolving deltas: 100% (1/1), done.
✅ เจอไฟล์แล้ว! ย้าย B0005.mat ออกมาเรียบร้อย
🧹 Cleanup complete.

✅ Setup Complete: พร้อมรัน Section ถัดไป


# Section 2: ตัวจัดการข้อมูล (Data Loader)

In [17]:
class BatteryDataLoader:
    def load_nasa_data(self, filepath="B0005.mat"):
        print(f"📂 กำลังอ่านและแปลงข้อมูลจาก {filepath} ...")

        try:
            mat = scipy.io.loadmat(filepath)
            data = mat['B0005'][0][0][0][0]
            dataset = []

            # ตัวแปรพักข้อมูล
            last_charge_props = {'time_at_4_15v': 0, 'min_voltage': 0}

            for i, cycle in enumerate(data):
                cycle_type = cycle['type'][0]

                # 1. เก็บข้อมูลตอนชาร์จ
                if cycle_type == 'charge':
                    try:
                        d = cycle['data'][0][0]
                        t = d['Time'][0]
                        v = d['Voltage_measured'][0]
                        high_v_idx = np.where(v >= 4.15)[0]

                        time_415 = t[high_v_idx[-1]] - t[high_v_idx[0]] if len(high_v_idx) > 1 else 0

                        last_charge_props = {
                            'time_at_4_15v': time_415,
                            'min_voltage': np.min(v)
                        }
                    except: pass

                # 2. เก็บข้อมูลตอนใช้งาน (Discharge)
                elif cycle_type == 'discharge':
                    try:
                        d = cycle['data'][0][0]
                        t = d['Time'][0]
                        v = d['Voltage_measured'][0]

                        dataset.append({
                            'discharge_time': t[-1] - t[0],
                            'max_voltage_discharge': np.max(v),
                            'min_voltage_charge': last_charge_props['min_voltage'],
                            'time_at_4_15v': last_charge_props['time_at_4_15v'],
                            'Capacity': d['Capacity'][0][0]
                        })
                    except: pass

            df = pd.DataFrame(dataset)

            # สร้าง Target RUL (อายุขัยที่เหลือ)
            eol_idx = df[df['Capacity'] < 1.4].index
            eol = eol_idx[0] if len(eol_idx) > 0 else len(df)

            # --- แก้ไขจุดที่ Error ---
            # คำนวณค่าก่อน (จะได้เป็น Series) แล้วค่อยสั่ง clip ทีหลัง
            df['RUL'] = eol - df.index
            df['RUL'] = df['RUL'].clip(lower=0)
            # ---------------------

            print(f"✅ โหลดข้อมูลสำเร็จ: {len(df)} รอบการชาร์จ")
            return df

        except Exception as e:
            print(f"❌ Error ในการอ่านไฟล์: {e}")
            return None

# เรียกใช้งานทันที
loader = BatteryDataLoader()
df = loader.load_nasa_data()

📂 กำลังอ่านและแปลงข้อมูลจาก B0005.mat ...
✅ โหลดข้อมูลสำเร็จ: 168 รอบการชาร์จ


# Section 3: สร้างและเทรน AI (Model Training)

In [18]:
class RULEstimator:
    def __init__(self):
        self.model = xgb.XGBRegressor(
            objective='reg:squarederror',
            n_estimators=100,
            max_depth=5
        )
        self.features = ['discharge_time', 'max_voltage_discharge', 'min_voltage_charge', 'time_at_4_15v']

    def train(self, df):
        if df is None: return

        X = df[self.features]
        y = df['RUL']

        print("🤖 กำลังเทรนโมเดล AI...")
        self.model.fit(X, y)
        print("✅ AI เทรนเสร็จเรียบร้อย พร้อมทำนาย!")

    def predict(self, df_row):
        # ฟังก์ชันทำนายรับข้อมูลเป็น DataFrame แถวเดียว
        return self.model.predict(df_row[self.features])[0]

# สร้างและเทรนทันที
ai_engine = RULEstimator()
ai_engine.train(df)

🤖 กำลังเทรนโมเดล AI...
✅ AI เทรนเสร็จเรียบร้อย พร้อมทำนาย!


# Section 4: ระบบคิดเงิน (Pricing Strategy)

In [19]:
class PricingSystem:
    def __init__(self, base_price=20.0):
        self.base_price = base_price

    def calculate_bill(self, ai_model, user_batt, station_batt):
        # 1. ให้ AI ประเมินสุขภาพแบต
        rul_user = ai_model.predict(user_batt)
        rul_station = ai_model.predict(station_batt)

        # 2. คำนวณราคาตามสูตร
        price_per_rul = 0.1
        rul_diff = rul_station - rul_user

        # คิดเงินเพิ่มถ้าแบตสถานีมี Voltage สูงกว่า (คุณภาพดีกว่า)
        volt_diff = (station_batt['max_voltage_discharge'].values[0] -
                     user_batt['max_voltage_discharge'].values[0]) * 2.0

        total = self.base_price + (rul_diff * price_per_rul) + volt_diff

        return {
            'RUL_User': rul_user,
            'RUL_Station': rul_station,
            'Base': self.base_price,
            'RUL_Cost': rul_diff * price_per_rul,
            'Quality_Cost': volt_diff,
            'Total': max(0, total) # ห้ามติดลบ
        }

# สร้างระบบคิดเงิน
pricing_bot = PricingSystem()
print("✅ ระบบคิดเงินพร้อมทำงาน")

✅ ระบบคิดเงินพร้อมทำงาน


# Section 5: ทดสอบจริง (Main Simulation)

In [20]:
if df is not None:
    print("\n" + "="*45)
    print("🔋 เริ่มต้นสถานการณ์จำลอง: ลูกค้ามาเปลี่ยนแบต")
    print("="*45)

    # --- จำลองเหตุการณ์ ---
    # 1. แบตลูกค้า (เก่า/เสื่อม) -> เอาข้อมูลแถวที่ 600 (ใกล้พัง)
    user_batt_data = df.iloc[[600]] if len(df) > 600 else df.iloc[[-1]]

    # 2. แบตสถานี (ใหม่/สด) -> เอาข้อมูลแถวที่ 5
    station_batt_data = df.iloc[[5]]

    # --- คำนวณบิล ---
    bill = pricing_bot.calculate_bill(ai_engine, user_batt_data, station_batt_data)

    # --- แสดงผลใบเสร็จ ---
    print(f"ผลการตรวจสภาพแบตเตอรี่:")
    print(f"   🔴 แบตเดิมของลูกค้า:   เหลืออายุใช้งาน {bill['RUL_User']:.1f} รอบ")
    print(f"   🟢 แบตใหม่ของสถานี:    เหลืออายุใช้งาน {bill['RUL_Station']:.1f} รอบ")
    print("-" * 45)
    print(f"🧾 รายละเอียดค่าบริการ:")
    print(f"   1. ค่าธรรมเนียมพื้นฐาน:    ${bill['Base']:.2f}")
    print(f"   2. ส่วนต่างอายุแบต (RUL): +${bill['RUL_Cost']:.2f}")
    print(f"   3. ส่วนต่างคุณภาพไฟ:      +${bill['Quality_Cost']:.2f}")
    print("=" * 45)
    print(f"💰 ยอดรวมที่ต้องชำระ:      ${bill['Total']:.2f}")
    print("=" * 45)

else:
    print("❌ ไม่สามารถรันได้ เนื่องจากโหลดข้อมูลไม่สำเร็จ")


🔋 เริ่มต้นสถานการณ์จำลอง: ลูกค้ามาเปลี่ยนแบต
ผลการตรวจสภาพแบตเตอรี่:
   🔴 แบตเดิมของลูกค้า:   เหลืออายุใช้งาน 0.0 รอบ
   🟢 แบตใหม่ของสถานี:    เหลืออายุใช้งาน 119.0 รอบ
---------------------------------------------
🧾 รายละเอียดค่าบริการ:
   1. ค่าธรรมเนียมพื้นฐาน:    $20.00
   2. ส่วนต่างอายุแบต (RUL): +$11.90
   3. ส่วนต่างคุณภาพไฟ:      +$-0.03
💰 ยอดรวมที่ต้องชำระ:      $31.87
